In [ ]:
def Gauss_smoothing(data, params):
    
    #for unique trips ??
    
    df = np.copy(data)
    
    SZ_1 = params['sz_1']
    SZ_2 = params['sz_2']
    SZ_CRIT = params['sz_crit']    
    
    lat_g1 = gaussian_filter1d(df[:, 0], np.sqrt(SZ_1))
    lon_g1 = gaussian_filter1d(df[:, 1], np.sqrt(SZ_1))
    lat_g2 = gaussian_filter1d(df[:, 0], np.sqrt(SZ_2))
    lon_g2 = gaussian_filter1d(df[:, 1], np.sqrt(SZ_2))

    lat_dif = df[1:,0] - df[:-1,0]
    lon_dif = df[1:,1] - df[:-1,1]

    lat_crit = np.append(np.abs(gaussian_filter1d(lat_dif, np.sqrt(SZ_CRIT)) / (1e-9 + gaussian_filter1d(np.abs(lat_dif), np.sqrt(SZ_CRIT)))),[0])
    lon_crit = np.append(np.abs(gaussian_filter1d(lon_dif, np.sqrt(SZ_CRIT)) / (1e-9 + gaussian_filter1d(np.abs(lon_dif), np.sqrt(SZ_CRIT)))),[0])           
            
    df[:, 0] = lat_g1 * lat_crit + lat_g2 * (1.0 - lat_crit)
    df[:, 1] = lon_g1 * lon_crit + lon_g2 * (1.0 - lon_crit)    
                       
    return df

In [1]:
def optimize(params):
    v_up_th, height_th, v_out_sigma, v_out_sigma = params
    
    score_list = []
    
    path = '/Users/vadimzubkov/Desktop/smartphone-decimeter-2022/train/2021-03-16-US-MTV-1/GooglePixel4XL'

    drive, phone = path.split('/')[-2:]


    gnss_df = pd.read_csv(f'{path}/device_gnss.csv')     

    gt_df = pd.read_csv(f'{path}/ground_truth.csv')

    utc, x_wls, v_wls, cov_x, cov_v = point_positioning(gnss_df)

    x_wls, v_wls, cov_x, cov_v = exclude_interpolate_outlier(x_wls, v_wls, cov_x, cov_v)

    x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, cov_x, cov_v, phone)

    llh_wls = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
    llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T


    llh_gs = Gauss_smoothing(llh_kf, {'sz_1' : sz_1, 'sz_2' : sz_2, 'sz_crit' : sz_crit})


    x_bl = gnss_df.groupby('TimeNanos')[
        ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']].mean().to_numpy()
    llh_bl = np.array(pm.ecef2geodetic(x_bl[:, 0], x_bl[:, 1], x_bl[:, 2])).T


    llh_gt = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
        
    score_bl = calc_score(llh_bl, llh_gt)
    score_wls = calc_score(llh_wls, llh_gt)
    score_kf = calc_score(llh_kf[:-1, :], llh_gt[:-1, :])
    score_gs = calc_score(llh_gs, llh_gt) 
    
    score_list.append(score_gs)

    mean_score = np.mean(score_list)
    
    return mean_score

In [2]:
space = [Real(0.5, 3.0, name='v_up_th'), 
         Integer(50, 300, name='height_th'), 
         Integer(1, 10, name='v_out_sigma'),
         Integer(20, 50, name='v_out_sigma')]

result = gp_minimize(optimize, space, n_calls=100, initial_point_generator='random')

NameError: name 'Real' is not defined

In [ ]:
print(f'best train score: {result.fun}') 

print(f'best params:\nsz_1: {result.x[0]}\nsz_2: {result.x[1]}\nsz_crit: {result.x[2]}')